In [3]:
import heartpy as hp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
pd.options.display.max_rows = 450

sample_rate = 240.0

In [4]:
# Todo: better tagging. Currently, not useful.

# The except blocks are placeholders, I want to see the errors right now and I am going to handle them later.
def preprocess(data, sample_rate=240.0):
    # Change values ouside possible range to min and max pulse value
    data = [0 if i <= 0 else (550 if i > 550 else i) for i in data]
    tag = ""
    try:
        data = hp.filter_signal(data, cutoff=15, sample_rate=sample_rate, order=4, filtertype='lowpass')
        tag += "Low"
    except ZeroDivisionError as e:
        print("Can't Divide by Zero -", e)
    try:
        data = hp.filter_signal(data, cutoff=.01, sample_rate=sample_rate, order=4, filtertype='highpass')
        tag += "High"
    except ZeroDivisionError as e:
        print("Can't Divide by Zero -", e)
    try:
        working_data, measures = hp.process(hrdata=data, sample_rate=sample_rate, report_time=False, bpmmin=0, bpmmax=550)
        # plot = hp.plotter(working_data, measures, show = True, title = 'Heart Rate Signal Peak Detection')
    except:
        return
    return working_data, measures, tag

In [16]:
waves = {}
for d in range(22):
    d += 1
    if(d < 10):
        d = '0' + str(d)
    else:
        d = str(d)
    data = hp.get_data('Data/Raw Data/Multiple Cath/X0' + d + '.txt', delim = ' ', column_name = 'AO')
    # print('X0' + d + '.txt')
    try:
        working_data, _, tag = preprocess(data, sample_rate)
        df = pd.DataFrame(working_data['hr'])
        waves['X0' + d] = df
    except:
        print('An error occurred with ' + 'X0' + d + '.txt')

/home/anrath/miniconda3/envs/biopeaks/lib/python3.7/site-packages/numpy/ma/core.py:5240: RuntimeWarning: Mean of empty slice.
  dtype=dtype, **kwargs)[()]
/home/anrath/miniconda3/envs/biopeaks/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3703: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)


In [19]:
waveMetrics = {}
def metricCalc(waves):
    for key, wave in waves.items():
        print(key)
        waveMetrics[key] = {}
        #identifying first 3 indexes
        beg = 0
        end = len(wave) - 1
        sysMaxi = wave.idxmax()[0]
        waveMetrics[key]['sysMaxi'] = sysMaxi

        #pp_pres
        pp_pres = sum(wave)

        #avg_sys_rise
        avg_sys_rise = wave[0:sysMaxi].mean()

        #sys_rise_area
        sys_rise_area = sum(wave[0:sysMaxi])

        #t_sys_rise
        t_sys_rise = sysMaxi

        #avg_dec
        avg_dec = wave[sysMaxi:end].mean()

        #t_dec
        t_dec = end - sysMaxi

        #dec_area
        dec_area = sum(wave[sysMaxi:end])

        #setting bounds for finding dicrotic notch and diastolic pressure
        diff = wave.diff()
        upperBound = round(sysMaxi + .3125*len(wave))

        #dicrotic notch
        for i in range(sysMaxi+1,upperBound):
            if diff.loc[i][0] > 0 and diff.loc[i+1][0] > 0:
                break
        dicNotch = i

        #diastolic pressure
        for i in range(dicNotch+1,upperBound):
            if diff.loc[i][0] < 0 and diff.loc[i+1][0] < 0:
                break
        diaP = i

        #avg_sys
        avg_sys = wave[beg:dicNotch].mean()

        #slope_sys
        slope_sys = (wave.loc[dicNotch][0] - wave[0]) / dicNotch

        #sys_area
        sys_area = sum(wave[0:dicNotch])

        #t_sys
        t_sys = dicNotch

        #avg_sys_dec
        avg_sys_dec = wave[sysMaxi:dicNotch].mean()

        #dn_sys
        dn_sys = wave.loc[sysMaxi][0] - wave.loc[dicNotch][0]

        #sys_dec_area
        sys_dec_area = sum(wave[sysMaxi:dicNotch])

        #t_sys_dec
        t_sys_dec = dicNotch - sysMaxi

        #avg_sys_dec_nodia
        avg_sys_dec_nodia = avg_sys_dec - wave.loc[diaP][0]

        #avg_sys_nodia
        avg_sys_nodia = avg_sys - wave.loc[diaP][0]

        #avg_sys_rise_nodia
        avg_sys_rise_nodia = avg_sys_rise - wave.loc[diaP][0]

        #avg_dec_nodia
        avg_dec_nodia = avg_dec - wave.loc[diaP][0]

        #slope_dia
        slope_dia = (wave.loc[end][0] - wave.loc[dicNotch][0]) / (end - dicNotch)

        #t_dia
        t_dia = end - dicNotch

        #avg_dia
        avg_dia = wave[dicNotch:end].mean()

        #dn_dia
        dn_dia = wave.loc[diaP][0] - wave.loc[dicNotch][0]

        #avg_sys_nodia
        avg_sys_nodia = avg_sys - wave.loc[diaP][0]


        

## Testing

In [20]:
metricCalc(waves)
waveMetrics[key]['sysMaxi']

X001
X002
X003
X004


KeyError: 1846

In [28]:
wave = waves['X004']
sysMaxi = wave.idxmax()[0]
print(sysMaxi)
diff = wave.diff()
upperBound = round(sysMaxi + .3125*len(wave)) # upperBound is greater than the length of X004 so this causes problems
print(upperBound)

#dicrotic notch
for i in range(sysMaxi+1,upperBound):
    if diff.loc[i][0] > 0 and diff.loc[i+1][0] > 0:
        break
dicNotch = i
print(dicNotch)

1632
2209


KeyError: 1846

In [26]:
diff
range(sysMaxi+1,upperBound)

range(1633, 2209)